In [ ]:
import numpy as np
import pandas as pd
import cv2
import os
import pickle
import matplotlib.pyplot as plt
path= r"C:\Users\HP\Desktop\task"



In [ ]:
train=pd.read_csv(path+r"\task"+r"\annotation.csv")


train_1=train.set_index('image')
train_name=set()
#test_name=set()
for index,row in train.iterrows():
    train_name.add(str(row['image']))


mat=train.as_matrix()

train_name=list(sorted(train_name))
train_name[0]

In [ ]:
X=[]
Y=[]
image_path="C:/Users/HP/Desktop/task/task/data/"


In [ ]:

for name in train_name:
    image=cv2.resize(cv2.imread(image_path+name+".jpg"),(90,90 ))
    X.append(list(image))
i=0
for name in train_name:
    Y.append(mat[i,1:])
    i+=1

X=np.asarray(X)




In [ ]:
np.save(path+"/Y.npy",Y)
np.save(path+"/X120.npy",X)


In [ ]:
#test_X=[]
#test=test.set_index('image')
#for index,row in test.iterrows():
#    image=cv2.imread(test_image_path+index)
#    test_X.append(image)
#test_X=np.array(test_X)
#np.save(path+"/train/test_X120.npy",test_X)


In [ ]:
X=np.load('C:/Users/HP/Desktop/task/X120.npy')


In [ ]:
Y=pd.DataFrame(Y)
a=Y.loc[7].iloc[0]
a

In [ ]:

Y=pd.DataFrame(Y)
for j in range(len(Y)):
    a=Y.loc[j].iloc[0]
    Y.loc[j]=a.replace("[","").replace("]","").replace("'","").replace(", "," ").replace("_","")



In [ ]:
z=list(Y[0])


In [ ]:
from sklearn.feature_extraction.text import CountVectorizer


vectorizer = CountVectorizer(lowercase=False)

matrix = vectorizer.fit_transform(z)

print(matrix.todense())
print(matrix.shape)

In [ ]:
from sklearn.model_selection import train_test_split
Xtrain, Xtest, Ytrain, Ytest = train_test_split(X,matrix, test_size = 0.2, random_state = 0)

In [ ]:
import numpy as np
from keras import layers
from keras import regularizers, optimizers
from keras.initializers import glorot_uniform
from keras.layers import Input, Dense, Activation, ZeroPadding2D, BatchNormalization, Flatten, Conv2D
from keras.layers import AveragePooling2D, MaxPooling2D, Dropout, GlobalMaxPooling2D, GlobalAveragePooling2D
from keras.models import Model,Sequential
from keras.preprocessing import image
from keras.utils import layer_utils
from keras.utils.data_utils import get_file
from keras.applications.imagenet_utils import preprocess_input

from IPython.display import SVG
from keras.utils.vis_utils import model_to_dot
from keras.utils import plot_model


import keras.backend as K
K.set_image_data_format('channels_last')


# Normalize image vectors
X_train = Xtrain/255.
X_test = Xtest/255.

# Reshape
Y_train = Ytrain.T
Y_test = Ytest.T

print ("number of training examples = " + str(X_train.shape[0]))
print ("number of test examples = " + str(X_test.shape[0]))
print ("X_train shape: " + str(X_train.shape))
print ("Y_train shape: " + str(Y_train.shape))



In [ ]:
def Model(input_shape):
    weight_decay = 1e-4;
    baseMapNum = 32
    model = Sequential()
    model.add(Conv2D(baseMapNum, (3,3), padding='same', kernel_regularizer=regularizers.l2(weight_decay), input_shape=(input_shape,input_shape,3)))
    model.add(Activation('relu'))
    model.add(BatchNormalization())
    model.add(Conv2D(baseMapNum, (3,3), padding='same', kernel_regularizer=regularizers.l2(weight_decay)))
    model.add(Activation('relu'))
    model.add(BatchNormalization())
    model.add(MaxPooling2D(pool_size=(2,2)))
    model.add(Dropout(0.2))

    model.add(Conv2D(2*baseMapNum, (3,3), padding='same', kernel_regularizer=regularizers.l2(weight_decay)))
    model.add(Activation('relu'))
    model.add(BatchNormalization())
    model.add(Conv2D(2*baseMapNum, (3,3), padding='same', kernel_regularizer=regularizers.l2(weight_decay)))
    model.add(Activation('relu'))
    model.add(BatchNormalization())
    model.add(MaxPooling2D(pool_size=(2,2)))
    model.add(Dropout(0.3))

    model.add(Conv2D(4*baseMapNum, (3,3), padding='same', kernel_regularizer=regularizers.l2(weight_decay)))
    model.add(Activation('relu'))
    model.add(BatchNormalization())
    model.add(Conv2D(4*baseMapNum, (3,3), padding='same', kernel_regularizer=regularizers.l2(weight_decay)))
    model.add(Activation('relu'))
    model.add(BatchNormalization())
    model.add(MaxPooling2D(pool_size=(2,2)))
    model.add(Dropout(0.4))

    model.add(Flatten())
    model.add(Dense(33, activation='softmax'))

    model.summary()
    
  
    
    return model


In [ ]:
Model1=Model(90)
Model1.compile(optimizer='adam',loss='categorical_crossentropy',metrics=['accuracy'])
from sklearn.metrics import f1_score
Model1.fit(Xtrain,Ytrain,epochs=20,batch_size=16,validation_data=(Xtest,Ytest),shuffle=True)
prediction=Model1.predict(Xtest,batch_size=16,verbose=1)
fscore=f1_score(YTest,prediction)
print(fscore)





In [ ]:
y=vectorizer.get_feature_name()
PREDICTION =pd.dataFrame(prediction,column=list(y))



